# Import

In [52]:
data_path='data/'
api_key='ehlVSIKSMiugJZHlii3sU7OKCexe4MIlnYenGric'
import pandas as pd 
import numpy as np

# Analysis

In [78]:
df = pd.read_csv(f"{data_path}epi_r.csv")
print(df[df['cognac/armagnac']==1]['cognac/armagnac'].sum())
for name, count in df.nunique().items():
    print(f"{name}: {count}")

91.0
title: 17736
rating: 8
calories: 1858
protein: 282
fat: 326
sodium: 2434
#cakeweek: 2
#wasteless: 2
22-minute meals: 2
3-ingredient recipes: 2
30 days of groceries: 2
advance prep required: 2
alabama: 2
alaska: 2
alcoholic: 2
almond: 2
amaretto: 2
anchovy: 2
anise: 2
anniversary: 2
anthony bourdain: 2
aperitif: 2
appetizer: 2
apple: 2
apple juice: 2
apricot: 2
arizona: 2
artichoke: 2
arugula: 2
asian pear: 2
asparagus: 2
aspen: 2
atlanta: 2
australia: 2
avocado: 2
back to school: 2
backyard bbq: 2
bacon: 2
bake: 2
banana: 2
barley: 2
basil: 2
bass: 2
bastille day: 2
bean: 2
beef: 2
beef rib: 2
beef shank: 2
beef tenderloin: 2
beer: 2
beet: 2
bell pepper: 2
berry: 2
beverly hills: 2
birthday: 2
biscuit: 2
bitters: 2
blackberry: 2
blender: 2
blue cheese: 2
blueberry: 2
boil: 2
bok choy: 2
bon appétit: 2
bon app��tit: 2
boston: 2
bourbon: 2
braise: 2
bran: 2
brandy: 2
bread: 2
breadcrumbs: 2
breakfast: 2
brie: 2
brine: 2
brisket: 2
broccoli: 2
broccoli rabe: 2
broil: 2
brooklyn: 2
br

The dataset have target feature:
```rating```.

There is also features: ```title, calories, protein, fat, sodium```



Dataset have 2 categorical features after the One-Hot Encoding conversion.

first categorical feature is:

Ingredients: ```almond, anchovy, apple, avocado, bacon```

second categorical feature is tags

The tags may contain:
* Recipe characteristics: ```#cakeweek, #wasteless, 22-minute meals, 3-ingredient recipes...```

* Geographical affiliation: ```alabama, arizona, boston, france, italy...```

* Dietary restrictions and features: ```dairy free, gluten free, vegan...```

* Cooking methods: ```bake, boil, grill, slow cooker...```

* Holidays and events: ```christmas, halloween, valentine's day...```

* Machinery and tools: ```blender, food processor, pressure cooker, microwave...```

кароче грубо говоря нужно с помощью апи проверять является ли признак ингредиентом, если да оставляем его в выборке 

for future needs

In [54]:
nutrients_list = [
    # Основные макронутриенты 
    "Total lipid (fat)",
    "Fatty acids, total saturated",
    "Fatty acids, total monounsaturated",
    "Fatty acids, total polyunsaturated",
    "Fatty acids, total trans",
    "Cholesterol",
    "Carbohydrate, by difference",
    "Sodium, Na", 
    "Fiber, total dietary",
    "Protein",
    "Sugars, Total",  
    "Energy", 
    "Vitamin A, RAE",  
    "Vitamin C, total ascorbic acid", 
    "Vitamin D (D2 + D3)",
    "Vitamin E (alpha-tocopherol)",
    "Vitamin K (phylloquinone)",
    "Thiamin",
    "Riboflavin",
    "Niacin",
    "Vitamin B-6",
    "Folate, total",
    "Vitamin B-12",
    
    # Минералы
    "Calcium, Ca",
    "Iron, Fe",
    "Phosphorus, P",
    "Magnesium, Mg",
    "Zinc, Zn",
    "Copper, Cu",
    "Manganese, Mn",
    "Selenium, Se",
    "Potassium, K",
    
    # Специфичные компоненты 
    "Ash",
    "Nitrogen",
    "Water",
    
    # Разные формы сахаров
    "Fructose",
    "Glucose",
    "Sucrose",
    "Galactose"
]


In [55]:
import requests
import json
def load_nutrients_cache(path_to_file):
    try:
        with open(path_to_file, 'r', encoding='utf-8') as f:
            try:
                return json.load(f)
            except:
                return {}
    except:
        return {}
def save_to_cache(path_to_file, ingredient, nutrient_dict, cache):
    cache[ingredient]=nutrient_dict
    with open(path_to_file, 'w', encoding='utf-8') as f:
        json.dump(cache, f, indent=4, ensure_ascii=False)

def get_nutrients_info(ingredient):
    ingredient_info=None
    params = {
        "query": f'"{ingredient}"',
        "api_key": api_key,
        "dataType": ["Foundation"]  
    }
    cache=load_nutrients_cache(f"{data_path}ingridient_nutr_cache.json")
    non_ingredient_cache=load_nutrients_cache(f"{data_path}not_ingridient_cache.json")
    if ingredient in cache:
        print(ingredient, "ok")
        return cache[ingredient]
    elif ingredient not in non_ingredient_cache:
        response = requests.get('https://api.nal.usda.gov/fdc/v1/foods/search', params=params)
        if response.status_code == 200:
            ingredient_info = response.json().get('foods')
            if ingredient_info and ingredient_info[0].get('score') < 200:
                ingredient_info=None
            if ingredient_info:
                ingredient_info=ingredient_info[0].get('foodNutrients')
                nutrient_df=pd.json_normalize(ingredient_info)[['nutrientName','unitName','value']]
                nutrient_dict=nutrient_df[nutrient_df['nutrientName'].isin(nutrients_list)].to_dict()       
                save_to_cache(f"{data_path}ingridient_nutr_cache.json",ingredient, nutrient_dict, cache)
    if not ingredient_info:
        save_to_cache(f"{data_path}not_ingridient_cache.json",ingredient, ingredient_info, non_ingredient_cache)
        print(ingredient, "not ok")
    return ingredient_info
def is_ingredient(ingredient):
    data=get_nutrients_info(ingredient)
    return True if data else False


## 2 lets sort columns in our dataset

In [56]:
features =[]
for column in df.columns:
    for ingredients in column.split("/"):
        if is_ingredient(ingredients):
            features.append(column)
            break
print (features)


title not ok
rating not ok
calories not ok
protein not ok
fat ok
sodium ok
#cakeweek not ok
#wasteless not ok
22-minute meals not ok
3-ingredient recipes not ok
30 days of groceries not ok
advance prep required not ok
alabama not ok
alaska not ok
alcoholic not ok
almond ok
amaretto not ok
anchovy not ok
anise not ok
anniversary not ok
anthony bourdain not ok
aperitif not ok
appetizer not ok
apple ok
apple juice ok
apricot ok
arizona not ok
artichoke not ok
arugula ok
asian pear not ok
asparagus ok
aspen not ok
atlanta not ok
australia not ok
avocado ok
back to school not ok
backyard bbq not ok
bacon ok
bake not ok
banana ok
barley ok
basil not ok
bass not ok
bastille day not ok
bean ok
beef ok
beef rib not ok
beef shank not ok
beef tenderloin ok
beer not ok
beet ok
bell pepper not ok
berry not ok
beverly hills not ok
birthday not ok
biscuit not ok
bitters not ok
blackberry ok
blender not ok
blue cheese not ok
blueberry ok
boil ok
bok choy ok
bon appétit not ok
bon app��tit not ok
bosto

## Now we get the cloumns for X and y

In [57]:
non_ingredients = [
    'fat',
    'sodium',
    "boil",        
    "braise",       
    "breakfast",   
    "broil",        
    "cookie",       
    "cookies",      
    "fat free",     
    "kosher",        
    "low fat",      
    "no sugar added", 
    "no-cook",   
    "pastry",     
    "pie",    
    "raw",          
    "summer",      
    "tart",       
    "winter",       
]
for value in non_ingredients:
    if value in features:
        features.remove(value)
print(features)

['almond', 'apple', 'apple juice', 'apricot', 'arugula', 'asparagus', 'avocado', 'bacon', 'banana', 'barley', 'bean', 'beef', 'beef tenderloin', 'beet', 'blackberry', 'blueberry', 'bok choy', 'bran', 'bread', 'broccoli', 'brussel sprout', 'bulgur', 'butter', 'buttermilk', 'cabbage', 'cantaloupe', 'carrot', 'cashew', 'cauliflower', 'celery', 'cheddar', 'cherry', 'chestnut', 'chicken', 'chickpea', 'cobbler/crumble', 'coconut', 'cod', 'corn', 'cottage cheese', 'crab', 'cranberry', 'cream cheese', 'cucumber', 'dill', 'egg', 'eggplant', 'feta', 'fig', 'fish', 'garlic', 'grains', 'grape', 'grapefruit', 'green onion/scallion', 'ham', 'hazelnut', 'hominy/cornmeal/masa', 'honeydew', 'hummus', 'kale', 'kiwi', 'lamb', 'leek', 'lentil', 'lettuce', 'low/no sugar', 'macadamia nut', 'mango', 'meat', 'melon', 'milk/cream', 'monterey jack', 'mozzarella', 'mushroom', 'mustard', 'nectarine', 'nut', 'oat', 'oatmeal', 'olive', 'onion', 'orange', 'orange juice', 'oyster', 'parmesan', 'pasta', 'pea', 'peach'

In [58]:
X=df[features]
y=df['rating']

In [59]:
print(X.shape) 
print(y.shape) 

X.sum().sort_values(ascending=False)

(20052, 139)
(20052,)


wheat/gluten-free    4906.0
onion                2238.0
tomato               2140.0
milk/cream           1995.0
egg                  1768.0
                      ...  
apple juice             6.0
cobbler/crumble         6.0
salsa                   6.0
bran                    3.0
hummus                  2.0
Length: 139, dtype: float64

отладка     

In [89]:
def ingr_check(ingredient):
    ingredient_info=None
    params = {
        "query": f'"{ingredient}"',
        "api_key": api_key,
        "dataType": ["Foundation"]  
    }
    response = requests.get('https://api.nal.usda.gov/fdc/v1/foods/search', params=params)
    if response.status_code == 200:
        ingredient_info = response.json().get('foods')
        print (ingredient_info[0].get('description'), ingredient_info[0].get('score'))
ingr_check("cheese")

Cheese, cheddar 187.77193


### now not edited funcs for grid searching 


это чисто препроцессинг, то что я сделал выше мб оформлю его так же шобы в пайплайн запихать 

In [61]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

#### Preprocessing 

In [ ]:
class TrainValidationTest(BaseEstimator, TransformerMixin):
        def __init__(self, test_size=0.2, random_state=21, stratified=True):
            self.test_size = test_size
            self.random_state = random_state
            self.stratified = stratified
        def split(self, X , y):
            stratify = y if self.stratified else None
            X_train, X_test, y_train, y_test =train_test_split(
                X, y, test_size=self.test_size, random_state=self.random_state, stratify=stratify
            )
            X_train, X_valid, y_train, y_valid =train_test_split(
                X, y, test_size=self.test_size/(1-self.test_size), random_state=self.random_state, stratify=stratify
            )
            return X_train, X_valid, X_test, y_train, y_valid, y_test

#### Model selection pipeline(need to edit)


In [ ]:
from copy import deepcopy
from itertools import product
from sklearn.metrics import get_scorer
def manual_grid_search(X_train, y_train, X_valid, y_valid, model_estimator,param_grid, cv, scoring='accuracy_score',  n_jobs=-1, error_score=np.nan):
    res=[]
    estimator=deepcopy(model_estimator)
    total_combinations = np.prod([len(v) for v in param_grid.values()])
    # get scorer for valid score
    scorer = get_scorer(scoring)
    for params in tqdm(list(product(*param_grid.values())), total=total_combinations, desc="GridSearch"):
        current_model = deepcopy(estimator)
        current_model.set_params(**(dict(zip(param_grid.keys(), params))))
        try:
            score=cross_val_score(
                current_model, X_train, y_train, cv=cv, scoring=scoring, n_jobs=n_jobs,error_score=error_score
            )

            current_model.fit(X_train, y_train)
            valid_score=scorer(current_model, X_valid, y_valid)

            res.append({
                **(dict(zip(param_grid.keys(), params))),
                'train_score': score.mean(),
                'valid_score': valid_score
            })


        except Exception as e:
            print(f"Error with params {params}: {str(e)}")
            res.append({
                **(dict(zip(param_grid.keys(), params))),
                'train_score': 'Na',
                'valid_score': 'Na'
            })
            continue
        
    return pd.DataFrame(res).sort_values('valid_score', ascending=False)

In [ ]:
class ModelSelection:
    def __init__ (self,estimators,estimators_name, grid_dict):
        self.estimarors=estimators
        self.estimarors_name=estimators_name
        self.grid_dict=grid_dict
        self.results=[]
    def choose(self,X_train, y_train, X_valid, y_valid, cv=2):
        """
        Method choose() takes X_train, y_train, X_valid, y_valid, optional cv, 
        and returns the name of the best model among all the models on the validation set
        """
        for estimator in self.estimarors:
            model_name= self.estimarors_name[self.estimarors.index(estimator)]
            grid_res=manual_grid_search(X_train,y_train,X_valid, y_valid,estimator,self.grid_dict[model_name] cv=cv)
            best_params = grid_res.drop(["train_score","valid_score"], axis=1).to_dict('records')[0]
            train_score = grid_res.head(1)['train_score'].item()
            best_valid_score = grid_res.head(1)['valid_score'].item()
            self.results.append({
                'model': model_name,
                'params': best_params,
                'valid_score': best_valid_score,
                'train_score': train_score
            })
            
            print(f"Best params: {best_params}")
            print(f"training accuracy: {train_score:.3f}")
            print(f"Best on validation set accuracy score: {best_valid_score:.3f}")
        self.results=pd.DataFrame(self.results)
        self.results.sort_values('valid_score', ascending=False,inplace=True)
        best_model_name=self.results.head(1)['model'].item()
        print(f"\nBest model: {best_model_name}")
    def best_results(self):
        return self.results[['model', 'params', 'valid_score']]
    def fit_score(self,X_comb,y_comb, X_test, y_test):
        best_model_name=self.results.head(1)['model'].item()
        best_estimator=self.estimators[self.estimators_name.index(best_model_name)]
        best_estimator.set_params(**(self.results.head(1)['params'].item()))
        best_estimator.fit(X_comb,y_comb)
        return ("the final accuracy score is :",accuracy_score(y_test, best_estimator.predict(X_test)))
        

#### Finalization
Takes an estimator.Method ```final_score()``` takes ```X_train, y_train, X_test, y_test``` and returns the accuracy

Method ```save_model()``` takes a path, saves the model to this path and prints that the model was successfully saved

In [65]:
from joblib import dump
class Finalize():
    def __init__(self, estimator):
        self.estimator = estimator
    def final_score(self,X_train, y_train, X_test, y_test):
        self.estimator.fit(X_train, y_train)
        score=accuracy_score(y_test, self.estimator.predict(X_test))
        print("Accuracy of the final model is",score)
        return score
    def save_model(self,filename):
        dump(self.estimator, f"{filename}.sav")

MAIN PROGGRAM LATER...